In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Set the hyperparameters for cross-validation
num_folds = 5
epochs = 15
batch_size = 32
image_size = 256
channels = 3

In [3]:
# Load the dataset
dataset_path = "dataset"
df = keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    shuffle=True,
    image_size=(image_size, image_size),
    batch_size=batch_size,
)

Found 16011 files belonging to 10 classes.


In [4]:
# Get the number of classes and class names
num_classes = len(df.class_names)
class_names = df.class_names

In [10]:
!pip install keras-tuner


     -------------------------------------- 176.1/176.1 kB 3.5 MB/s eta 0:00:00


In [ ]:
# Define the model building function for Keras Tuner
def build_model(hp):
    model = keras.Sequential([
        layers.experimental.preprocessing.Rescaling(1.0 / 255),
        layers.Conv2D(
            hp.Int("conv1_units", min_value=32, max_value=256, step=32),
            kernel_size=(3, 3),
            activation="relu",
            input_shape=(image_size, image_size, channels),
        ),
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Add more convolutional and pooling layers as desired
        layers.Flatten(),
        layers.Dense(
            hp.Int("dense_units", min_value=64, max_value=256, step=64),
            activation="relu",
        ),
        layers.Dense(num_classes, activation="softmax"),
    ])
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

fold_histories = []
fold_accuracies = []
fold_losses = []

for fold in range(num_folds):
    print(f"Training and evaluating fold {fold+1}/{num_folds}")
    
    # Split the dataset into training and validation sets for this fold
    validation_size = len(df) // num_folds
    val_ds = df.take(validation_size).skip(fold * validation_size)
    train_ds = df.skip(validation_size).take((num_folds - 1) * validation_size)
    
    # Preprocess and augment the training data
    data_augmentation = keras.Sequential([
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        layers.experimental.preprocessing.RandomRotation(0.2),
    ])
    train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
    
    # Create the model instance for this fold using Keras Tuner
    tuner = Hyperband(
        build_model,
        objective="val_accuracy",
        max_epochs=epochs,
        directory="keras_tuner",
        project_name="tomato_classification",
    )
    tuner.search(train_ds, validation_data=val_ds, epochs=epochs)
    
    # Retrieve the best model from the search
    best_model = tuner.get_best_models(num_models=1)[0]
    
    # Train the best model on the full training set for this fold
    best_model.fit(train_ds, epochs=epochs, verbose=1)

Training and evaluating fold 1/5

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
128               |128               |conv1_units
256               |256               |dense_units
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


In [ ]:
 # Evaluate the model on the validation set
    _, accuracy = model.evaluate(val_ds)
    fold_histories.append(history)
    fold_accuracies.append(accuracy)
    fold_losses.append(history.history["val_loss"][-1])

In [ ]:
# Calculate and print the average accuracy and loss across folds
average_accuracy = np.mean(fold_accuracies)
average_loss = np.mean(fold_losses)
print(f"\nAverage Accuracy: {average_accuracy}")
print(f"Average Loss: {average_loss}")

In [ ]:
# Plot the training and validation accuracy and loss curves for each fold
plt.figure(figsize=(12, 6))
for i, history in enumerate(fold_histories):
    plt.subplot(1, 2, 1)
    plt.plot(range(epochs), history.history["accuracy"], label=f"Fold {i+1}")
    plt.title("Training Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    
    plt.subplot(1, 2, 2)
    plt.plot(range(epochs), history.history["val_accuracy"], label=f"Fold {i+1}")
    plt.title("Validation Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    
plt.tight_layout()
plt.legend()
plt.show()